In [2]:
#finds site map of entire domain
from usp.tree import sitemap_tree_for_homepage

#for saving and loading objects
import pickle

#for accessing url html
import requests
import urllib.request, urllib.parse, urllib.error
from urllib.request import Request, urlopen

#for html parsing
from bs4 import BeautifulSoup

#anxillary functions/methods
import numpy as np
import pandas as pd
import time
import random
import datetime
from dateutil.tz import tzutc, tzoffset
from tqdm.notebook import tqdm



# Sitemap

In [ ]:
tree = sitemap_tree_for_homepage('https://hongkongfp.com/');

In [ ]:
urls = []
dates = []
for page in tree.all_pages():
    urls.append(page.url)
    dates.append(page.last_modified)
    
#transpose
zippedList =list(zip(urls,dates))

#format into dataframe
df = pd.DataFrame(zippedList, columns = ['urls' ,'date'])

#drop nan for date published
df = df.dropna()
        
pickle.dump(df, open("../Pickles/HKFP_All.p", "wb" ) )


# Data Scrape

In [9]:
#LOAD DATASET OF ALL SCRAPED URLS FROM SCMP
with open("../Pickles/HKFP_All.p", 'rb') as f:
    df = pickle.load(f)
          
#reset df index
df.reset_index(drop = True)

#find articles with "Articles" to exclude photos/video pages
articles_only_urls = []
for i in df.urls:
    if '2018' in i or '2019' in i or '2020' in i:
        articles_only_urls.append(i)
        
author = []
title = []
date = []
summary = []
link = []
broken = []

len(articles_only_urls)

15388

In [10]:
#using BS4 to request each video html data
for i in tqdm(articles_only_urls[2000:3000]):
    try:
        
        #time.sleep(1) 
        req = Request(str(i), headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'lxml')

        by = soup.find("a", {"class": "url fn n"})
        t = soup.find("h1", {"class": "entry-title"})
        d = soup.find("time", {"class": "entry-date published"})
        summ = soup.find("div",{"class":"entry-content"}).select_one("p")


        if by and t and d:
            author.append(by.text)
            title.append(t.text.strip())
            date.append(d.text)
            summary.append(summ.text.strip().replace(u'\xa0', u' '))
            link.append(i)

            if len(link)% 1000 == 0:
                print("Dataset contains " + str(len(link)) + " entries.")
    except:
        print("URL number " + str(urls.index(i)) + " is broken: " + i )
        broken.append(i)
        pass

Dataset contains 1000 entries.
Dataset contains 2000 entries.



In [13]:
df = pd.DataFrame(list(zip(title, author, date, summary, link)), 
               columns =['Title', 'Author', 'Date', 'Summary','Link']) 
    

with open('../Pickles/HkfpDF.pkl', 'wb') as f:
    pickle.dump(df, f)

# Data Analysis

In [ ]:
df = pd.read_pickle(r'../Pickles/SCMP_Opinions.pkl')
SCMPEditorial = df[df['Author'] == 'SCMP Editorial']
SCMPEditorial = SCMPEditorial.reset_index(drop=True)

In [ ]:
df

In [5]:
title

['In Pictures: Hundreds protest against Tuen Mun’s ‘dancing aunties’ as police deploy pepper spray',
 'Traces of coronavirus detected in Hong Kong patient’s dog but no evidence of infection, says gov’t dept.',
 'China censors report about how authorities hid coronavirus genome sequence test results for 14 days',
 'Community film screenings across Hong Kong mark 9 months since first anti-extradition law protest clashes',
 'Hundreds commemorate 15-year-old Hong Kong student who died last September',
 'In Pictures: New Hong Kong water park to make a splash in 2018',
 '‘I want to die for HK’ football coach to stay until 2018',
 'China’s LeEco inks exclusive deal to show 2018 football World Cup in Hong Kong',
 'HKFP Lens: ‘We are Hong Kong’ – memories of the 2018 World Cup qualifiers',
 'China’s population to grow 45 million by 2020: Plan',
 'Trial for Mong Kok protesters Ray Wong and Edward Leung to begin in 2018']